Connessione a spotipy

In [3]:
!pip install spotipy 

  Obtaining dependency information for spotipy from https://files.pythonhosted.org/packages/ea/35/304e456a471128aa4a776243558f43aee3444731ef8fc9bc8c351fddfdd8/spotipy-2.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for redis>=3.5.3 from https://files.pythonhosted.org/packages/c5/d1/19a9c76811757684a0f74adc25765c8a901d67f9f6472ac9d57c844a23c8/redis-5.0.8-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/255.6 kB ? eta -:--:--
   ----------------- ---------------------- 112.6/255.6 kB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 255.6/255.6 kB 3.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json
import os 
from saved_tracks_extractor import get_audio_features_and_analysis

# Configura le tue credenziali Spotify
client_id = os.getenv('YOUR_CLIENT_ID')
client_secret = os.getenv('YOUR_CLIENT_SECRET')
redirect_uri = os.getenv('YOUR_REDIRECT_URI')

scope = "user-library-read playlist-read-private"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope=scope))

SpotifyOauthError: No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.

In [2]:
# Initialize an empty list to store the results
results = []

# Set the batch size (max 50 for Spotify API)
batch_size = 5

# Get the total number of saved tracks
total_tracks = sp.current_user_saved_tracks(limit=1)['total']
print(f"Total tracks: {total_tracks}")

total_tracks = 50
# Loop through batches of saved tracks
for offset in range(0, total_tracks, batch_size):
    tracks = sp.current_user_saved_tracks(limit=batch_size, offset=offset)
    track_ids = [item['track']['id'] for item in tracks['items']]
    print(f"Track IDs: {track_ids}")    
    # Apply the get_audio_features_and_analysis function
    track_data = get_audio_features_and_analysis(track_ids, sp)
    # Append the result to the list
    results.append(track_data)

# Print or process the results as needed
for result in results:
    print("---")
    print(json.dumps(result, indent=4))

Total tracks: 9715
Track IDs: ['5vmjHtzvHeX2HYmWNSFqP6', '51VWrOnuL89xwoazeOZyXd', '4lee8gdN6r99KkY77YVHye', '5t9i10vYdDzA7rsEPW3lKf', '44UT5iLVHmgy1IXmOqiQdn']
Track IDs: ['0V31InZ8QrZ4bFHeZRXvXG', '09JdPiM1uQJz4iTrXLDmlo', '10whYsE8x94pqVdBguPJL5', '4qZWNigbOxdCnlgLSqEXj4', '4UriOPbcRKgEA3liG0lqE4']
Track IDs: ['4rFm4cgZhCsGaPNbbEc0G4', '1Scdm7tCv8Vu0yUeFRs4Iv', '1fjitT9XjUeZMP646nsiTL', '6T5fW3JC57UIGeYzwx91hY', '0FMLxnU3BE0VAvzqK2yIGR']
Track IDs: ['1hyOZrfiLof7tbqf5pJjta', '2n3FPgSY1SKMf3FdtyteRA', '2TtIKR20Mc66se6kGpmErP', '6F0bWpG7jhl0Hv92ZQnRHx', '4eNFa7cFWUv7koVWjjZYOi']
Track IDs: ['57m0xwokoN3J49tso8tGDJ', '21SdDged8mWRNvDnwPzxH1', '2qNpmmWewO58DmBHH9i0ug', '2W10cBw9jdMsyQLHFiOPr4', '0T4njSTUzEEBWT33eyXEFQ']
Track IDs: ['58l6rUrdGNOtLRsX93w40N', '6wVViUl2xSRoDK2T7dMZbR', '4jfsySxBIePLrNq5FYYjyv', '6FTUpZS85JMQzFEQ7wOqVK', '1SIXMGcsAeA6sNxaY1EG3O']
Track IDs: ['3NYm3ZimuJNb0Z0px5OEwO', '27wbCVE2JA72qLZVt7pTuF', '40oMDrqiESAbKzfnK6avpB', '0O9havTTUkI5F5LitVgfYH', '2bIQ3AcOR8iJ

In [ ]:
import pickle
import json
import os
import time

def update_pickle(new_data, pickle_file='spotify_data.pickle'):
    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as f:
            existing_data = pickle.load(f)
        for key in new_data:
            if key in existing_data:
                existing_data[key].extend(new_data[key])
            else:
                existing_data[key] = new_data[key]
    else:
        existing_data = new_data
    
    with open(pickle_file, 'wb') as f:
        pickle.dump(existing_data, f)

# Set the batch size (max 50 for Spotify API)
batch_size = 50

# Get the total number of saved tracks
total_tracks = sp.current_user_saved_tracks(limit=1)['total']
print(f"Total tracks: {total_tracks}")
total_tracks = 200

# Uncomment the next line if you want to limit the number of tracks for testing
# total_tracks = 5

# Start timer for the entire process
start_time_total = time.time()

# Loop through batches of saved tracks
for offset in range(0, total_tracks, batch_size):
    # Start timer for this batch
    start_time_batch = time.time()

    tracks = sp.current_user_saved_tracks(limit=batch_size, offset=offset)
    track_ids = [item['track']['id'] for item in tracks['items']]
    print(f"Processing track IDs: {track_ids}")    
    
    # Apply the get_audio_features_and_analysis function
    track_data = get_audio_features_and_analysis(track_ids, audio_analysis=True)
    
    # Update the pickle file with this batch of data
    update_pickle(track_data)
    
    print(f"Processed and saved tracks {offset} to {min(offset+batch_size, total_tracks)}")
    
    # Optional: print the data for this batch
    print("---")
    print(json.dumps(track_data, indent=4))

    # End timer for this batch
    end_time_batch = time.time()
    print(f"Time taken for this batch: {end_time_batch - start_time_batch:.2f} seconds")

# End timer for the entire process
end_time_total = time.time()
print(f"Total time taken: {end_time_total - start_time_total:.2f} seconds")

print("All data processed and saved.")

# If you want to read and display all the data at the end:
with open('spotify_data.pickle', 'rb') as f:
    all_data = pickle.load(f)

print("\nAll data from pickle file:")
print(json.dumps(all_data, indent=4))


In [ ]:
import pickle

with open('spotify_data.pickle', 'rb') as f:
    pickle_data = pickle.load(f)

print(pickle_data.keys())
for key in pickle_data:
    print(f"{key}: {len(pickle_data[key])}")


In [ ]:
# import data from pickle file
import pickle
with open('spotify_data.pickle', 'rb') as f:
    pickle_data = pickle.load(f)

# Convert the data to a pandas DataFrame
import pandas as pd
df = pd.DataFrame(pickle_data)
df.head()
len(df)
df.head()

## CLustering

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

def perform_kmeans_clustering(df, features, max_clusters=10, optimal_k=None):
    """
    Perform K-means clustering on the given DataFrame.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame
    features (list): List of feature column names to use for clustering
    max_clusters (int): Maximum number of clusters to consider for elbow method
    optimal_k (int): Optimal number of clusters. If None, user should determine from elbow curve.
    
    Returns:
    pandas.DataFrame: Original DataFrame with added 'Cluster' column
    """
    
    # Normalize the features
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(df[features])

    # Determine optimal number of clusters (elbow method)
    inertias = []
    for k in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(normalized_features)
        inertias.append(kmeans.inertia_)

    # Plot elbow curve
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, max_clusters + 1), inertias, marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')
    plt.title('Elbow Method')
    plt.show()

    if optimal_k is None:
        print("Please determine the optimal number of clusters from the elbow curve.")
        return df

    # Apply K-means with optimal number of clusters
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    cluster_labels = kmeans.fit_predict(normalized_features)

    # Add cluster labels to the original dataset
    df_with_clusters = df.copy()
    df_with_clusters['Cluster'] = cluster_labels

    # Visualize results (using first two features)
    plt.figure(figsize=(10, 6))
    plt.scatter(df[features[0]], df[features[1]], c=cluster_labels, cmap='viridis')
    plt.xlabel(features[0])
    plt.ylabel(features[1])
    plt.title('K-means Clustering Results')
    plt.show()

    return df_with_clusters

# Example usage:
# features = ['energy', 'danceability', 'tempo']
# result_df = perform_kmeans_clustering(df, features, optimal_k=8)



In [ ]:
perform_kmeans_clustering(df, ['energy', 'danceability', 'tempo'], optimal_k=10)

Le altre cose da importare oltre ai saved tracks

## Postgres


In [ ]:
import psycopg2
import json
from psycopg2.extras import Json

# Database connection parameters
db_params = {
    "host": "localhost",
    "database": "your_database_name",
    "user": "your_username",
    "password": "your_password"
}

# Assume 'results' is your list of JSON objects, one for each song

# Create a table in your PostgreSQL database (run this once)
def create_table():
    conn = psycopg2.connect(**db_params)
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS songs (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255),
        artist VARCHAR(255),
        danceability FLOAT,
        energy FLOAT,
        key INTEGER,
        loudness FLOAT,
        mode INTEGER,
        speechiness FLOAT,
        acousticness FLOAT,
        instrumentalness FLOAT,
        liveness FLOAT,
        valence FLOAT,
        tempo FLOAT,
        duration_ms INTEGER,
        time_signature INTEGER,
        segments INTEGER,
        bars INTEGER,
        beats INTEGER,
        sections INTEGER,
        tatums INTEGER,
        full_data JSONB
    )
    """)
    conn.commit()
    cur.close()
    conn.close()

# Insert data into the database
def insert_songs(songs):
    conn = psycopg2.connect(**db_params)
    cur = conn.cursor()

    for song in songs:
        cur.execute("""
        INSERT INTO songs (
            name, artist, danceability, energy, key, loudness, mode,
            speechiness, acousticness, instrumentalness, liveness,
            valence, tempo, duration_ms, time_signature, segments,
            bars, beats, sections, tatums, full_data
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            song['name'], song['artist'], song['danceability'], song['energy'],
            song['key'], song['loudness'], song['mode'], song['speechiness'],
            song['acousticness'], song['instrumentalness'], song['liveness'],
            song['valence'], song['tempo'], song['duration_ms'], song['time_signature'],
            song['segments'], song['bars'], song['beats'], song['sections'],
            song['tatums'], Json(song)
        ))

    conn.commit()
    cur.close()
    conn.close()

# Main execution
if __name__ == "__main__":
    # Create the table (run this once)
    create_table()

    # Insert the songs
    insert_songs(results)  # 'results' is your list of song JSONs

    print("Data inserted successfully!")


# SQlite

In [ ]:
import sqlite3
import json

# Database connection parameters
db_params = {
    "database": "your_database_name.db"  # SQLite uses a file-based database
}

# Assume 'results' is your list of JSON objects, one for each song

# Create a table in your SQLite database (run this once)
def create_table():
    conn = sqlite3.connect(**db_params)
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS songs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        artist TEXT,
        danceability REAL,
        energy REAL,
        key INTEGER,
        loudness REAL,
        mode INTEGER,
        speechiness REAL,
        acousticness REAL,
        instrumentalness REAL,
        liveness REAL,
        valence REAL,
        tempo REAL,
        duration_ms INTEGER,
        time_signature INTEGER,
        segments INTEGER,
        bars INTEGER,
        beats INTEGER,
        sections INTEGER,
        tatums INTEGER,
        full_data TEXT
    )
    """)
    conn.commit()
    cur.close()
    conn.close()

# Insert data into the database
def insert_songs(songs):
    conn = sqlite3.connect(**db_params)
    cur = conn.cursor()

    for song in songs:
        cur.execute("""
        INSERT INTO songs (
            name, artist, danceability, energy, key, loudness, mode,
            speechiness, acousticness, instrumentalness, liveness,
            valence, tempo, duration_ms, time_signature, segments,
            bars, beats, sections, tatums, full_data
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            song['name'], song['artist'], song['danceability'], song['energy'],
            song['key'], song['loudness'], song['mode'], song['speechiness'],
            song['acousticness'], song['instrumentalness'], song['liveness'],
            song['valence'], song['tempo'], song['duration_ms'], song['time_signature'],
            song['segments'], song['bars'], song['beats'], song['sections'],
            song['tatums'], json.dumps(song)  # Store the JSON as a string
        ))

    conn.commit()
    cur.close()
    conn.close()

# Main execution
if __name__ == "__main__":
    # Create the table (run this once)
    create_table()

    # Insert the songs
    insert_songs(results)  # 'results' is your list of song JSONs

    print("Data inserted successfully!")
